<a href="https://colab.research.google.com/github/LanceMeister/SEO-Audit-Google-Search-Console-for-SEOs/blob/main/SEO_Audit_Google_Search_Console_Machine_Learning_for_SEOs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive 
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


# A Hands-on Introduction to Machine Learning for SEOs

Here is technical plan to generate the training dataset:

1. Extract: we will walk through my code to connect to Google Search Console and pull CTR data
2. Transform: then, we will fetch the pages titles and meta descriptions and normalize/clean them
3. Load: finally, we will populate a Google Sheet with the pages, titles, meta descriptions and CTRs

In [ ]:
spreadsheetName="GSC Data" #@param {type:"string"}
sitename="sc-domain:meisteraffiliateshop.com" #@param {type:"string"}
client_id='google_secret.json' #@param {type:"string"}


## Extracting from Google Search Console

First, there is some setup to download a client_id.json file our Python code can use to connect securely to Google Search Console.

1. Activate Search Console API in Compute Engine  https://console.cloud.google.com/apis/api/webmasters.googleapis.com/overview?project=&folder=&organizationId=
2. Create New Credentials / Help me choose (Search Console API, Other UI, User data)  https://console.cloud.google.com/apis/credentials/wizard?api=iamcredentials.googleapis.com&project=
3. Download client_id.json

In [ ]:
# Next, we need to upload the file
from google.colab import files

files.upload()

Saving google_secret.json to google_secret.json


{'google_secret.json': b'{"installed":{"client_id":"826988042811-vae6ham0l4a6n8kh94qcrqkhn0n5jrbo.apps.googleusercontent.com","project_id":"fluid-outcome-353518","auth_uri":"https://accounts.google.com/o/oauth2/auth","token_uri":"https://oauth2.googleapis.com/token","auth_provider_x509_cert_url":"https://www.googleapis.com/oauth2/v1/certs","client_secret":"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","redirect_uris":["http://localhost"]}}'}

In [ ]:
# Go to Runtime > run after
!pip install git+https://github.com/joshcarty/google-searchconsole

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/joshcarty/google-searchconsole to /tmp/pip-req-build-cswrya2f
  Running command git clone -q https://github.com/joshcarty/google-searchconsole /tmp/pip-req-build-cswrya2f
  Created wheel for searchconsole: filename=searchconsole-0.0.3-py3-none-any.whl size=8925 sha256=bb6fe9615daa029727917e67b3237b176f2addac6af8270279f55c5772d8907c
  Stored in directory: /tmp/pip-ephem-wheel-cache-6mc05ldq/wheels/c0/54/65/805cab909323977d11a22ab51aca484859347ee6a5fc8e2fef
Successfully built searchconsole


In [ ]:
# run once
import searchconsole
#Create the account connection. 
try:
	account = searchconsole.authenticate(flow="console",client_config='/content/drive/MyDrive/config/client_secret_395713215546-kmpj3c3r57e5f08dmrmd36js8u2m5av6.apps.googleusercontent.com.json',credentials='/content/drive/MyDrive/config/buoyant-zodiac-328808-e9f3add2cb89.json')
except:
	account = searchconsole.authenticate(flow="console",client_config='/content/drive/MyDrive/config/client_secret_395713215546-kmpj3c3r57e5f08dmrmd36js8u2m5av6.apps.googleusercontent.com.json',serialize='/content/drive/MyDrive/config/buoyant-zodiac-328808-e9f3add2cb89.json')


Let's get our Search Console data

In [ ]:
webproperty = account[sitename]

In [ ]:
#let's build a pandas dataframe with the search console data
import pandas as pd
%load_ext google.colab.data_table

def get_search_console_data(webproperty, days=-7):
  if webproperty is not None:
    query = webproperty.query.range(start='today', days=days).dimension('date', 'page', 'query', 'country', 'device')

    r = query.get()
    df = pd.DataFrame(r.rows)
    return df

  print("Web property doesn't exist, please select a valid one from this list")
  print(account.webproperties)

  return None


The google.colab.data_table extension is already loaded. To reload it, use:
  %reload_ext google.colab.data_table


In [ ]:
df = get_search_console_data(webproperty)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         12 non-null     object
 1   page         12 non-null     object
 2   query        12 non-null     object
 3   country      12 non-null     object
 4   device       12 non-null     object
 5   clicks       12 non-null     int64 
 6   impressions  12 non-null     int64 
 7   ctr          12 non-null     int64 
 8   position     12 non-null     int64 
dtypes: int64(4), object(5)
memory usage: 992.0+ bytes


In [ ]:
df.head()

,date,page,query,country,device,clicks,impressions,ctr,position
0,2022-06-12,https://www.meisteraffiliateshop.com/de/kinder...,kinderschaukel draußen,deu,TABLET,0,1,0,11
1,2022-06-13,https://www.meisteraffiliateshop.com/smart-hom...,87102ls,cyp,DESKTOP,0,3,0,1
2,2022-06-14,https://www.meisteraffiliateshop.com/de/kinder...,schaukel draußen,esp,MOBILE,0,1,0,85
3,2022-06-14,https://www.meisteraffiliateshop.com/de/kinder...,schaukel drinnen,deu,DESKTOP,0,1,0,67
4,2022-06-14,https://www.meisteraffiliateshop.com/smart-hom...,hue e14,ven,MOBILE,0,1,0,80


In [ ]:
pages = list(set(df.page))
len(pages)

3

## Extracting SEO Audit

In [ ]:
try:
  from seoanalyzer import analyze
except:
  !pip install git+https://github.com/LanceMeister/python-seo-analyzer
  from seoanalyzer import analyze

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/LanceMeister/python-seo-analyzer to /tmp/pip-req-build-jn4w2wcq
  Running command git clone -q https://github.com/LanceMeister/python-seo-analyzer /tmp/pip-req-build-jn4w2wcq
  Created wheel for pyseoanalyzer: filename=pyseoanalyzer-4.0.5-py3-none-any.whl size=29098 sha256=b4431cbb3eb2ceca1e5d0dfaf631f1bfae069fb2d072ee0d5c5e45500aff8923
  Stored in directory: /tmp/pip-ephem-wheel-cache-84d1v93e/wheels/54/8e/b6/7d938bf6194dbad9e0b7ff16bbc68e578e8a8080ee2fdb0c96
Successfully built pyseoanalyzer


In [ ]:
output = analyze('https://www.meisteraffiliateshop.com', analyze_headings=True, analyze_extra_tags=True) 
seo_df= pd.DataFrame(output['pages'])
seo_df.head()

,page,title,description,word_count,keywords,bigrams,trigrams,warnings,content_hash,headings,additional_info
0,https://www.meisteraffiliateshop.com,meister affiliate shop,"welcome to meister affiliate shop, here you ca...",628,"[(36, cart), (36, add), (16, apple), (11, prod...","{'meister affiliate': 2, 'affiliate shop': 2, ...","{'meister affiliate shop': 2, 'affiliate shop ...",[Description is too short (less than 140 chara...,471dac0a221ce4cc5667501f97e86f874bbf1500,{'h2': [' Foldable outdoor camp...,"{'title': ['Meister Affiliate Shop'], 'meta_de..."
1,https://www.meisteraffiliateshop.com/,meister affiliate shop,"welcome to meister affiliate shop, here you ca...",628,"[(36, cart), (36, add), (16, apple), (11, prod...","{'meister affiliate': 2, 'affiliate shop': 2, ...","{'meister affiliate shop': 2, 'affiliate shop ...",[Description is too short (less than 140 chara...,2d76cdacf7014676d1d0fa8485e7f4b393a49975,{'h2': [' Foldable outdoor camp...,"{'title': ['Meister Affiliate Shop'], 'meta_de..."
2,https://www.meisteraffiliateshop.com/apple-pro...,apple products and accessories,explore new apple products or accessories for ...,203,"[(16, apple), (10, products), (9, cart), (9, a...","{'apple products': 6, 'products and': 4, 'and ...","{'apple products and': 4, 'products and access...",[Description is too short (less than 140 chara...,bb1e43ae17446566950afa4cb3254c58ec9e4bd4,{'h2': [' 2021 Apple TV HD (32G...,"{'title': ['Apple Products And Accessories'], ..."
3,https://www.meisteraffiliateshop.com/fitness/,fitness,find all your health & fitness equipment right...,101,"[(6, products), (6, fitness)]","{'fitness seasonal': 1, 'seasonal shopping': 1...","{'fitness seasonal shopping': 1, 'seasonal sho...",[Title tag is too short (less than 10 characte...,1b01b0c1e69b939a796cbdeb0ec52bc75b8dccb2,{'h2': [' PowerBlock Adjustable...,"{'title': ['Fitness'], 'meta_desc': ['Find all..."
4,https://www.meisteraffiliateshop.com/garden/,garden,"browse our garden essentials like grills, pool...",169,"[(6, products), (5, cart), (5, add)]","{'garden seasonal': 1, 'seasonal shopping': 1,...","{'garden seasonal shopping': 1, 'seasonal shop...",[Title tag is too short (less than 10 characte...,48082d5f6dcb2dca124793046c1c83f216346cc9,{'h2': [' Set of 2 Graphite Sun...,"{'title': ['Garden'], 'meta_desc': ['Browse ou..."


In [ ]:
seo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   page             205 non-null    object
 1   title            205 non-null    object
 2   description      205 non-null    object
 3   word_count       205 non-null    int64 
 4   keywords         205 non-null    object
 5   bigrams          205 non-null    object
 6   trigrams         205 non-null    object
 7   warnings         205 non-null    object
 8   content_hash     204 non-null    object
 9   headings         205 non-null    object
 10  additional_info  205 non-null    object
dtypes: int64(1), object(10)
memory usage: 17.7+ KB


In [ ]:
seo_df.dropna().head()

,page,title,description,word_count,keywords,bigrams,trigrams,warnings,content_hash,headings,additional_info
0,https://www.meisteraffiliateshop.com,meister affiliate shop,"welcome to meister affiliate shop, here you ca...",628,"[(36, cart), (36, add), (16, apple), (11, prod...","{'meister affiliate': 2, 'affiliate shop': 2, ...","{'meister affiliate shop': 2, 'affiliate shop ...",[Description is too short (less than 140 chara...,471dac0a221ce4cc5667501f97e86f874bbf1500,{'h2': [' Foldable outdoor camp...,"{'title': ['Meister Affiliate Shop'], 'meta_de..."
1,https://www.meisteraffiliateshop.com/,meister affiliate shop,"welcome to meister affiliate shop, here you ca...",628,"[(36, cart), (36, add), (16, apple), (11, prod...","{'meister affiliate': 2, 'affiliate shop': 2, ...","{'meister affiliate shop': 2, 'affiliate shop ...",[Description is too short (less than 140 chara...,2d76cdacf7014676d1d0fa8485e7f4b393a49975,{'h2': [' Foldable outdoor camp...,"{'title': ['Meister Affiliate Shop'], 'meta_de..."
2,https://www.meisteraffiliateshop.com/apple-pro...,apple products and accessories,explore new apple products or accessories for ...,203,"[(16, apple), (10, products), (9, cart), (9, a...","{'apple products': 6, 'products and': 4, 'and ...","{'apple products and': 4, 'products and access...",[Description is too short (less than 140 chara...,bb1e43ae17446566950afa4cb3254c58ec9e4bd4,{'h2': [' 2021 Apple TV HD (32G...,"{'title': ['Apple Products And Accessories'], ..."
3,https://www.meisteraffiliateshop.com/fitness/,fitness,find all your health & fitness equipment right...,101,"[(6, products), (6, fitness)]","{'fitness seasonal': 1, 'seasonal shopping': 1...","{'fitness seasonal shopping': 1, 'seasonal sho...",[Title tag is too short (less than 10 characte...,1b01b0c1e69b939a796cbdeb0ec52bc75b8dccb2,{'h2': [' PowerBlock Adjustable...,"{'title': ['Fitness'], 'meta_desc': ['Find all..."
4,https://www.meisteraffiliateshop.com/garden/,garden,"browse our garden essentials like grills, pool...",169,"[(6, products), (5, cart), (5, add)]","{'garden seasonal': 1, 'seasonal shopping': 1,...","{'garden seasonal shopping': 1, 'seasonal shop...",[Title tag is too short (less than 10 characte...,48082d5f6dcb2dca124793046c1c83f216346cc9,{'h2': [' Set of 2 Graphite Sun...,"{'title': ['Garden'], 'meta_desc': ['Browse ou..."


## Merge dataframes



In [ ]:
merged_df=pd.merge(df, seo_df, how="left", on="page")

In [ ]:
merged_df.head()

,date,page,query,country,device,clicks,impressions,ctr,position,title,description,word_count,keywords,bigrams,trigrams,warnings,content_hash,headings,additional_info
0,2022-06-12,https://www.meisteraffiliateshop.com/de/kinder...,kinderschaukel draußen,deu,TABLET,0,1,0,11,"gartenschaukel, kinderschaukel für drinnen ode...","gartenschaukel, kinderschaukel outdoor oder in...",312,"[(9, schaukel), (5, kinder)]","{'gartenschaukel kinderschaukel': 3, 'kindersc...","{'gartenschaukel kinderschaukel für': 3, 'kind...",[Description is too short (less than 140 chara...,64f3e2e66070b87b7ab04f2080dab3b8e06fdea4,"{'h1': ['Gartenschaukel, Kinderschaukel für dr...","{'title': ['Gartenschaukel, Kinderschaukel für..."
1,2022-06-13,https://www.meisteraffiliateshop.com/smart-hom...,87102ls,cyp,DESKTOP,0,3,0,1,philips hue white and colour ambiance e14 doub...,personalised lighting: choose from 16 million ...,253,"[(10, hue), (7, philips), (7, light), (7, home...","{'philips hue': 7, 'hue white': 3, 'white and'...","{'philips hue white': 3, 'hue white and': 3, '...",[Missing og:image],cbe210412b5b3c9a0753cbc0e058bd82cbcb910d,{'h1': ['Philips Hue White and Colour Ambiance...,{'title': ['Philips Hue White and Colour Ambia...
2,2022-06-14,https://www.meisteraffiliateshop.com/de/kinder...,schaukel draußen,esp,MOBILE,0,1,0,85,"gartenschaukel, kinderschaukel für drinnen ode...","gartenschaukel, kinderschaukel outdoor oder in...",312,"[(9, schaukel), (5, kinder)]","{'gartenschaukel kinderschaukel': 3, 'kindersc...","{'gartenschaukel kinderschaukel für': 3, 'kind...",[Description is too short (less than 140 chara...,64f3e2e66070b87b7ab04f2080dab3b8e06fdea4,"{'h1': ['Gartenschaukel, Kinderschaukel für dr...","{'title': ['Gartenschaukel, Kinderschaukel für..."
3,2022-06-14,https://www.meisteraffiliateshop.com/de/kinder...,schaukel drinnen,deu,DESKTOP,0,1,0,67,"gartenschaukel, kinderschaukel für drinnen ode...","gartenschaukel, kinderschaukel outdoor oder in...",312,"[(9, schaukel), (5, kinder)]","{'gartenschaukel kinderschaukel': 3, 'kindersc...","{'gartenschaukel kinderschaukel für': 3, 'kind...",[Description is too short (less than 140 chara...,64f3e2e66070b87b7ab04f2080dab3b8e06fdea4,"{'h1': ['Gartenschaukel, Kinderschaukel für dr...","{'title': ['Gartenschaukel, Kinderschaukel für..."
4,2022-06-14,https://www.meisteraffiliateshop.com/smart-hom...,hue e14,ven,MOBILE,0,1,0,80,philips hue white and colour ambiance e14 doub...,personalised lighting: choose from 16 million ...,253,"[(10, hue), (7, philips), (7, light), (7, home...","{'philips hue': 7, 'hue white': 3, 'white and'...","{'philips hue white': 3, 'hue white and': 3, '...",[Missing og:image],cbe210412b5b3c9a0753cbc0e058bd82cbcb910d,{'h1': ['Philips Hue White and Colour Ambiance...,{'title': ['Philips Hue White and Colour Ambia...


In [ ]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12 entries, 0 to 11
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   date             12 non-null     object
 1   page             12 non-null     object
 2   query            12 non-null     object
 3   country          12 non-null     object
 4   device           12 non-null     object
 5   clicks           12 non-null     int64 
 6   impressions      12 non-null     int64 
 7   ctr              12 non-null     int64 
 8   position         12 non-null     int64 
 9   title            12 non-null     object
 10  description      12 non-null     object
 11  word_count       12 non-null     int64 
 12  keywords         12 non-null     object
 13  bigrams          12 non-null     object
 14  trigrams         12 non-null     object
 15  warnings         12 non-null     object
 16  content_hash     12 non-null     object
 17  headings         12 non-null     obje

In [ ]:
merged_df.dropna().head()

,date,page,query,country,device,clicks,impressions,ctr,position,title,description,word_count,keywords,bigrams,trigrams,warnings,content_hash,headings,additional_info
0,2022-06-12,https://www.meisteraffiliateshop.com/de/kinder...,kinderschaukel draußen,deu,TABLET,0,1,0,11,"gartenschaukel, kinderschaukel für drinnen ode...","gartenschaukel, kinderschaukel outdoor oder in...",312,"[(9, schaukel), (5, kinder)]","{'gartenschaukel kinderschaukel': 3, 'kindersc...","{'gartenschaukel kinderschaukel für': 3, 'kind...",[Description is too short (less than 140 chara...,64f3e2e66070b87b7ab04f2080dab3b8e06fdea4,"{'h1': ['Gartenschaukel, Kinderschaukel für dr...","{'title': ['Gartenschaukel, Kinderschaukel für..."
1,2022-06-13,https://www.meisteraffiliateshop.com/smart-hom...,87102ls,cyp,DESKTOP,0,3,0,1,philips hue white and colour ambiance e14 doub...,personalised lighting: choose from 16 million ...,253,"[(10, hue), (7, philips), (7, light), (7, home...","{'philips hue': 7, 'hue white': 3, 'white and'...","{'philips hue white': 3, 'hue white and': 3, '...",[Missing og:image],cbe210412b5b3c9a0753cbc0e058bd82cbcb910d,{'h1': ['Philips Hue White and Colour Ambiance...,{'title': ['Philips Hue White and Colour Ambia...
2,2022-06-14,https://www.meisteraffiliateshop.com/de/kinder...,schaukel draußen,esp,MOBILE,0,1,0,85,"gartenschaukel, kinderschaukel für drinnen ode...","gartenschaukel, kinderschaukel outdoor oder in...",312,"[(9, schaukel), (5, kinder)]","{'gartenschaukel kinderschaukel': 3, 'kindersc...","{'gartenschaukel kinderschaukel für': 3, 'kind...",[Description is too short (less than 140 chara...,64f3e2e66070b87b7ab04f2080dab3b8e06fdea4,"{'h1': ['Gartenschaukel, Kinderschaukel für dr...","{'title': ['Gartenschaukel, Kinderschaukel für..."
3,2022-06-14,https://www.meisteraffiliateshop.com/de/kinder...,schaukel drinnen,deu,DESKTOP,0,1,0,67,"gartenschaukel, kinderschaukel für drinnen ode...","gartenschaukel, kinderschaukel outdoor oder in...",312,"[(9, schaukel), (5, kinder)]","{'gartenschaukel kinderschaukel': 3, 'kindersc...","{'gartenschaukel kinderschaukel für': 3, 'kind...",[Description is too short (less than 140 chara...,64f3e2e66070b87b7ab04f2080dab3b8e06fdea4,"{'h1': ['Gartenschaukel, Kinderschaukel für dr...","{'title': ['Gartenschaukel, Kinderschaukel für..."
4,2022-06-14,https://www.meisteraffiliateshop.com/smart-hom...,hue e14,ven,MOBILE,0,1,0,80,philips hue white and colour ambiance e14 doub...,personalised lighting: choose from 16 million ...,253,"[(10, hue), (7, philips), (7, light), (7, home...","{'philips hue': 7, 'hue white': 3, 'white and'...","{'philips hue white': 3, 'hue white and': 3, '...",[Missing og:image],cbe210412b5b3c9a0753cbc0e058bd82cbcb910d,{'h1': ['Philips Hue White and Colour Ambiance...,{'title': ['Philips Hue White and Colour Ambia...


## Keywords in title

https://stackoverflow.com/questions/31806695/when-to-use-which-fuzz-function-to-compare-2-strings
https://chairnerd.seatgeek.com/fuzzywuzzy-fuzzy-string-matching-in-python/
https://github.com/seatgeek/fuzzywuzzy

In [ ]:
!pip install fuzzywuzzy[speedup]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50 kB 5.2 MB/s 
  Created wheel for python-levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149857 sha256=82a0cd3daf6f8b095e7e9af06b2b243c9e3aa76385fe031c19c83a23b02bfd7f
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built python-levenshtein


In [ ]:
from fuzzywuzzy import fuzz

In [ ]:
#remove empty rows
df = merged_df.dropna()

In [ ]:
#df.apply(lambda row: fuzz.partial_ratio(row["query"], row["title"]), axis = 1)
df["query_in_title"] = df.apply(lambda row: fuzz.partial_ratio(row["query"], row["title"]), axis=1)
# shift column 'query_in_title' to tenth position
tenth_column = df.pop('query_in_title')
# insert column using insert(position,column_name,first_column) function
df.insert(10, 'query_in_title', tenth_column)
df

,date,page,query,country,device,clicks,impressions,ctr,position,title,query_in_title,description,word_count,keywords,bigrams,trigrams,warnings,content_hash,headings,additional_info
0,2022-06-12,https://www.meisteraffiliateshop.com/de/kinder...,kinderschaukel draußen,deu,TABLET,0,1,0,11,"gartenschaukel, kinderschaukel für drinnen ode...",77,"gartenschaukel, kinderschaukel outdoor oder in...",312,"[(9, schaukel), (5, kinder)]","{'gartenschaukel kinderschaukel': 3, 'kindersc...","{'gartenschaukel kinderschaukel für': 3, 'kind...",[Description is too short (less than 140 chara...,64f3e2e66070b87b7ab04f2080dab3b8e06fdea4,"{'h1': ['Gartenschaukel, Kinderschaukel für dr...","{'title': ['Gartenschaukel, Kinderschaukel für..."
1,2022-06-13,https://www.meisteraffiliateshop.com/smart-hom...,87102ls,cyp,DESKTOP,0,3,0,1,philips hue white and colour ambiance e14 doub...,29,personalised lighting: choose from 16 million ...,253,"[(10, hue), (7, philips), (7, light), (7, home...","{'philips hue': 7, 'hue white': 3, 'white and'...","{'philips hue white': 3, 'hue white and': 3, '...",[Missing og:image],cbe210412b5b3c9a0753cbc0e058bd82cbcb910d,{'h1': ['Philips Hue White and Colour Ambiance...,{'title': ['Philips Hue White and Colour Ambia...
2,2022-06-14,https://www.meisteraffiliateshop.com/de/kinder...,schaukel draußen,esp,MOBILE,0,1,0,85,"gartenschaukel, kinderschaukel für drinnen ode...",69,"gartenschaukel, kinderschaukel outdoor oder in...",312,"[(9, schaukel), (5, kinder)]","{'gartenschaukel kinderschaukel': 3, 'kindersc...","{'gartenschaukel kinderschaukel für': 3, 'kind...",[Description is too short (less than 140 chara...,64f3e2e66070b87b7ab04f2080dab3b8e06fdea4,"{'h1': ['Gartenschaukel, Kinderschaukel für dr...","{'title': ['Gartenschaukel, Kinderschaukel für..."
3,2022-06-14,https://www.meisteraffiliateshop.com/de/kinder...,schaukel drinnen,deu,DESKTOP,0,1,0,67,"gartenschaukel, kinderschaukel für drinnen ode...",75,"gartenschaukel, kinderschaukel outdoor oder in...",312,"[(9, schaukel), (5, kinder)]","{'gartenschaukel kinderschaukel': 3, 'kindersc...","{'gartenschaukel kinderschaukel für': 3, 'kind...",[Description is too short (less than 140 chara...,64f3e2e66070b87b7ab04f2080dab3b8e06fdea4,"{'h1': ['Gartenschaukel, Kinderschaukel für dr...","{'title': ['Gartenschaukel, Kinderschaukel für..."
4,2022-06-14,https://www.meisteraffiliateshop.com/smart-hom...,hue e14,ven,MOBILE,0,1,0,80,philips hue white and colour ambiance e14 doub...,71,personalised lighting: choose from 16 million ...,253,"[(10, hue), (7, philips), (7, light), (7, home...","{'philips hue': 7, 'hue white': 3, 'white and'...","{'philips hue white': 3, 'hue white and': 3, '...",[Missing og:image],cbe210412b5b3c9a0753cbc0e058bd82cbcb910d,{'h1': ['Philips Hue White and Colour Ambiance...,{'title': ['Philips Hue White and Colour Ambia...
5,2022-06-14,https://www.meisteraffiliateshop.com/smart-hom...,philips hue 470,are,MOBILE,0,1,0,10,philips hue white and colour ambiance e14 doub...,80,personalised lighting: choose from 16 million ...,253,"[(10, hue), (7, philips), (7, light), (7, home...","{'philips hue': 7, 'hue white': 3, 'white and'...","{'philips hue white': 3, 'hue white and': 3, '...",[Missing og:image],cbe210412b5b3c9a0753cbc0e058bd82cbcb910d,{'h1': ['Philips Hue White and Colour Ambiance...,{'title': ['Philips Hue White and Colour Ambia...
6,2022-06-14,https://www.meisteraffiliateshop.com/smart-hom...,philips hue 470,est,MOBILE,0,1,0,8,philips hue white and colour ambiance e14 doub...,80,personalised lighting: choose from 16 million ...,253,"[(10, hue), (7, philips), (7, light), (7, home...","{'philips hue': 7, 'hue white': 3, 'white and'...","{'philips hue white': 3, 'hue white and': 3, '...",[Missing og:image],cbe210412b5b3c9a0753cbc0e058bd82cbcb910d,{'h1': ['Philips Hue White and Colour Ambiance...,{'title': ['Philips Hue White and Colour Ambia...
7,2022-06-14,https://www.meisteraffiliateshop.com/smart-hom...,philips hue e14,vnm,MOBILE,0,1,0,87,philips hue white a

In [ ]:
de_df = df.query('country == "deu"')
de_df

,date,page,query,country,device,clicks,impressions,ctr,position,title,query_in_title,description,word_count,keywords,bigrams,trigrams,warnings,content_hash,headings,additional_info
0,2022-06-12,https://www.meisteraffiliateshop.com/de/kinder...,kinderschaukel draußen,deu,TABLET,0,1,0,11,"gartenschaukel, kinderschaukel für drinnen ode...",77,"gartenschaukel, kinderschaukel outdoor oder in...",312,"[(9, schaukel), (5, kinder)]","{'gartenschaukel kinderschaukel': 3, 'kindersc...","{'gartenschaukel kinderschaukel für': 3, 'kind...",[Description is too short (less than 140 chara...,64f3e2e66070b87b7ab04f2080dab3b8e06fdea4,"{'h1': ['Gartenschaukel, Kinderschaukel für dr...","{'title': ['Gartenschaukel, Kinderschaukel für..."
3,2022-06-14,https://www.meisteraffiliateshop.com/de/kinder...,schaukel drinnen,deu,DESKTOP,0,1,0,67,"gartenschaukel, kinderschaukel für drinnen ode...",75,"gartenschaukel, kinderschaukel outdoor oder in...",312,"[(9, schaukel), (5, kinder)]","{'gartenschaukel kinderschaukel': 3, 'kindersc...","{'gartenschaukel kinderschaukel für': 3, 'kind...",[Description is too short (less than 140 chara...,64f3e2e66070b87b7ab04f2080dab3b8e06fdea4,"{'h1': ['Gartenschaukel, Kinderschaukel für dr...","{'title': ['Gartenschaukel, Kinderschaukel für..."


In [ ]:
# df.head()
# df.to_csv(r"/content/drive/MyDrive/Colab Notebooks/seoAudit/output/ml_gsc_seo.csv")

## Populating our Google Sheet

In [ ]:
#https://pypi.org/project/gspread-pandas/


In [ ]:
!pip install gspread-pandas
!pip install --upgrade google-auth[reauth]
#https://github.com/aiguofer/gspread-pandas/pull/47

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gspread
    Found existing installation: gspread 3.4.2
    Uninstalling gspread-3.4.2:
      Successfully uninstalled gspread-3.4.2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pyu2f: filename=pyu2f-0.1.5-py3-none-any.whl size=39404 sha256=b5c125a39785a2ebec97806338037e22487a85fab93966886554b48b0dfa3632
  Stored in directory: /root/.cache/pip/wheels/b3/84/c6/ee8093bf96e2224aca3a9bfa074aa3c86c39208f91055fd60f
Successfully built pyu2f


In [ ]:
from google.colab import auth
auth.authenticate_user()
import google.auth
import gspread

In [ ]:
from gspread_pandas import Spread, Client

In [ ]:
import google.auth

creds, project = google.auth.default()


In [ ]:
spread = Spread(spreadsheetName, creds=creds)
spread


<gspread_pandas.spread.Spread - 'User: 'k.lancemeister@gmail.com', Spread: 'GSC Data', Sheet: 'Sheet1''>

In [ ]:
#https://gspread-pandas.readthedocs.io/en/latest/getting_started.html

In [ ]:
# Display available worksheets
spread.sheets

[<Worksheet 'Sheet1' id:0>, <Worksheet 'Client' id:1120219873>]

In [ ]:
# Save DataFrame to worksheet 'Client', create it first if it doesn't exist
spread.df_to_sheet(df, index=False, sheet='Client', start='A1', replace=True)

In [ ]:
spread.sheets

[<Worksheet 'Sheet1' id:0>, <Worksheet 'Client' id:1120219873>]